In [1]:
!pip install ultralytics
!pip install validators

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import cv2
from torchvision import transforms
from torchvision.models import efficientnet_v2_m as effnetv2m
from google.colab import drive

In [3]:
# ======
# device
# ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# ===============
# batch inference
# ===============
def batch_infer(
    images, cls_custom=None, ci_custom=0.1, top_classes=5,
    model_detector="yolov5x", weight_detector=None,
    model_classifier="effnetv2m", weight_classifier=None
):
    # check image quantity
    if len(images) != 2:
        raise ValueError("only image quantity '2' supported for batch inference.")

    # check models
    if model_detector != "yolov5x":
        raise ValueError("only model 'yolov5x' supported for detector.\n")

    if model_classifier != "effnetv2m":
        raise ValueError("only model 'effnetv2m' is supported for classifier.")

    # load models
    detector = torch.hub.load(
        "ultralytics/yolov5",
        "custom",
        path=weight_detector,
        force_reload=True
    )
    detector = detector.to(device)

    classifier = effnetv2m()
    classifier.load_state_dict(torch.load(weight_classifier, map_location=device), strict=False)

    # detect and classify objects
    clses, probs = [], []

    for image in images:
        # load image
        with open(image, "rb") as f:
            image = f.read()
            arr = np.asarray(bytearray(image), dtype=np.uint8)
            image = cv2.imdecode(arr, -1)

        # check image
        if image is None:
            raise Exception(f"no valid image read via 'cv2.imread()'.\n")

        # detect objects
        output = detector(image)
        output = output.pandas().xyxy[0]
        output = output[output["name"] == cls_custom]

        # check output
        if output.empty:
            raise Exception(f"no '{cls_custom}' detected with confidence interval >= '{ci_custom}'.")
        else:
            if output.shape[0] >= 2:
                output = output.loc[[output["confidence"].idxmax()]]

        # calculate bounding box
        xmin, xmax, ymin, ymax = output["xmin"], output["xmax"], output["ymin"], output["ymax"]

        # crop image
        image = image[int(ymin):int(ymax), int(xmin):int(xmax)]

        # configure transformation
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

        # transform image
        image = transform(image)
        image = image.unsqueeze(0)

        with torch.no_grad():
            output = torch.nn.functional.softmax(classifier(image), dim=1)
        output = torch.topk(output, top_classes)

        cls, prob = output.indices, output.values

        clses.append(cls.squeeze())
        probs.append(prob.squeeze())

    cls_common = [cls for cls in clses[0] if cls in clses[1]]

    indices = []
    for i in range(len(images)):
        inds = []
        for cls in cls_common:
            for ind, val in enumerate(clses[i]):
                if val == cls:
                    inds.append(ind)
        indices.append(inds)

    prob_common = []
    for i in range(len(cls_common)):
        prob_common.append(probs[0][indices[0][i]] * probs[1][indices[1][i]])

    output = {
        "class": cls_common,
        "joint probability": prob_common
    }

    output = pd.DataFrame(output)

    return output

In [5]:
# ==========
# load files
# ==========
drive.mount('/content/drive')

path_project = "/content/drive/MyDrive/Animal-Detector"

path_weight = os.path.join(path_project, "weights")
weight_detector = os.path.join(path_weight, "yolov5x.pt")
weight_classifier = os.path.join(path_weight, "effnetv2m.pth")

image = os.path.join(path_project, "image.jpg")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# ======================
# test - batch inference
# ======================
output = batch_infer(
    images=[image, image], cls_custom="Insecta", top_classes=20,
    weight_detector=weight_detector, weight_classifier=weight_classifier
)
print(f"output\n{output}")

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-7-13 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 322 layers, 86254162 parameters, 0 gradients
Adding AutoShape... 


output
         class    joint probability
0  tensor(462)  tensor(2.17979e-06)
1  tensor(958)  tensor(2.22342e-06)
2  tensor(539)  tensor(2.30369e-06)
3  tensor(997)  tensor(2.47873e-06)
4  tensor(389)  tensor(2.13701e-06)
5  tensor(325)  tensor(2.09881e-06)
6  tensor(313)  tensor(2.08638e-06)
7  tensor(945)  tensor(1.98766e-06)
